In [1]:
import os
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [31]:
def visualize_ransac_ground_removal(session, frame):
    file_name = os.path.join(f'sessions/frames_{session:02d}', f'frame_{frame:05d}.npy')
    
    # Create an Open3D PointCloud object
    point_cloud_o3d = o3d.geometry.PointCloud()
    point_cloud_array = np.load(file_name)
    point_cloud_o3d.points = o3d.utility.Vector3dVector(point_cloud_array)

    # Segment the plane
    plane_model, inliers = point_cloud_o3d.segment_plane(distance_threshold=0.3,
                                                         ransac_n=3,
                                                         num_iterations=2000)
    
    [a, b, c, d] = plane_model
    print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

    # Find points below the plane
    all_points = point_cloud_np
    plane_eq = all_points.dot(plane_model[:3]) + d
    below_plane = np.where(plane_eq < 0)[0]

    # Combine inliers and below plane points
    combined_inliers = np.union1d(inliers, below_plane)

    # Create inlier and outlier clouds
    inlier_cloud = point_cloud_o3d.select_by_index(inliers)
    inlier_cloud.paint_uniform_color([1.0, 0, 0])  # Red color
    below_plain_cloud = point_cloud_o3d.select_by_index(below_plane)
    below_plain_cloud.paint_uniform_color([1.0, 1.0, 0])  # Yellow color 
    outlier_cloud = point_cloud_o3d.select_by_index(combined_inliers, invert=True)

    # Visualize
    o3d.visualization.draw_geometries([outlier_cloud, inlier_cloud, below_plain_cloud])


In [35]:
def ransac_ground_removal(session, frame):
    file_name = os.path.join(f'sessions/frames_{session:02d}', f'frame_{frame:05d}.npy')
    
    # Create an Open3D PointCloud object
    point_cloud_o3d = o3d.geometry.PointCloud()
    point_cloud_array = np.load(file_name)
    point_cloud_o3d.points = o3d.utility.Vector3dVector(point_cloud_array)

    # Segment the plane
    plane_model, inliers = point_cloud_o3d.segment_plane(distance_threshold=0.3,
                                                         ransac_n=3,
                                                         num_iterations=2000)
    
    [a, b, c, d] = plane_model
    print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

    # Find points below the plane
    all_points = point_cloud_array
    plane_eq = all_points.dot(plane_model[:3]) + d
    below_plane = np.where(plane_eq < 0)[0]

    # Combine inliers and below plane points
    combined_inliers = np.union1d(inliers, below_plane)

    outlier_cloud = point_cloud_o3d.select_by_index(combined_inliers, invert=True)

    non_ground_points_array = np.asarray(outlier_cloud.points)
    np.save(file_name, non_ground_points_array)

In [36]:
ransac_ground_removal(0,0)

Plane equation: -0.02x + 0.01y + 1.00z + 1.69 = 0
